In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_Michelin_3-star_restaurants'
html=requests.get(url).text
sopa=bs(html, 'html.parser')

In [3]:
tabla=sopa.find_all('table', class_='wikitable sortable')[0]
tbody=tabla.find('tbody')
registros=tbody.find_all('tr')[1]
datos=registros.find_all('td')
#registros.text

In [4]:
colum_names=['location','restaurant','chef','awarded_year']
df_restaurants=pd.DataFrame(columns=colum_names)


tablas=sopa.find_all('table', class_='wikitable sortable')

for tabla in tablas:
   
    tbody=tabla.find('tbody')
    filas=tbody.find_all('tr')
    
    for i in range(1,len(filas)-1):
        
        fila = filas[i]
        datos=fila.find_all('td')    
        registro={'location': datos[0].text.split('\n')[0],'restaurant': datos[1].text.split('\n')[0],
                  'chef': datos[2].text.split('[')[0],'awarded_year': datos[3].text.split('[')[0]} 
        
        df_restaurants=df_restaurants.append(registro, ignore_index=True)


In [5]:
df_restaurants=df_restaurants.reindex(columns=['restaurant', 'location','chef', 'awarded_year'])
df_restaurants.head()

,restaurant,location,chef,awarded_year
0,Hof van Cleve,Kruishoutem,Peter Goossens,2005
1,Zilte,Antwerp,Viki Geunes\n,2021
2,Xin Rong Ji,Beijing,Zhang Yong (Founder),2020
3,King's Joy,Beijing,Gary Yin\n,2021
4,Geranium,Copenhagen,Rasmus Kofoed,2016


In [6]:
df_restaurants.shape

(124, 4)

In [7]:
url ='https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105&displayCurrency=EUR'
html=requests.get(url).text
sopa=bs(html, 'html.parser')

In [9]:
container=sopa.find_all('div', id='page_container')[0]
tabla=container.find('table')
filas=tabla.find_all('tr')

colum_names=['country','net_salary']
df_salarys=pd.DataFrame(columns=colum_names)

for fila in filas:
    
    columnas=fila.find_all('td')
    net_salary=columnas[3].text.replace('\n','').replace(',','').replace('€','').strip()
    registro={'country': columnas[1].text.replace('\n',''),'net_salary': net_salary}        
    df_salarys=df_salarys.append(registro, ignore_index=True)
    
df_salarys.head()

,country,net_salary
0,Switzerland,5931.31
1,Singapore,4357.28
2,Australia,4092.05
3,United Arab Emirates,3799.26
4,United States,3755.59


In [10]:
##crear la cadena de conexion para conectarnos a mysql(le indicamos a que base de datos nos conectamos)
import src 
conn_ = src.get_conn_string('./password.txt','michelin')
###incluir el txt del password en gitignore

In [11]:
from sqlalchemy import create_engine
cursor=create_engine(conn_)


df_restaurants.to_sql(name='chefs', con=cursor, if_exists='replace', index=False)
df_salarys.to_sql(name='salaries', con=cursor, if_exists='replace', index=False)

106